In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# read in previously collected data
schools = pd.read_csv("Data/fbsSchoolData.csv") # season, team, id, mascot, abbrev, conference, division
playCalls = pd.read_csv("Data/fbsPlayCallData.csv") #season, team, conference, totalPlays, passingPlays, rushingPlays
winLoss = pd.read_csv("Data/fbsWinLossData.csv") #season, team, id, mascot, abbrev, conference, division, games, wins, losses, ties

# merge datasets together
merge_1 = pd.merge(schools, playCalls, on = ["season", "team"], how = "left")
merge_2 = pd.merge(merge_1, winLoss, on = ["season", "id"], how = "left")

# preview dataset
merge_2.head()

,season,team_x,id,mascot_x,abbreviation_x,conference_x,division_x,conference_y,totalPlays,passingPlays,rushingPlays,team_y,mascot_y,abbreviation_y,conference,division_y,games,wins,losses,ties
0,2004,Air Force,2005,Falcons,AFA,Mountain West,NaN,Mountain West,147.0,33.0,110.0,Air Force,Falcons,AFA,Mountain West,NaN,11,5,6,0
1,2004,Akron,2006,Zips,AKR,Mid-American,East,Mid-American,131.0,70.0,55.0,Akron,Zips,AKR,Mid-American,East,11,6,5,0
2,2004,Alabama,333,Crimson Tide,ALA,SEC,West,SEC,677.0,224.0,434.0,Alabama,Crimson Tide,ALA,SEC,West,11,6,5,0
3,2004,Arizona,12,Wildcats,ARIZ,Pac-10,NaN,Pac-12,681.0,314.0,354.0,Arizona,Wildcats,ARIZ,Pac-10,NaN,11,3,8,0
4,2004,Arizona State,9,Sun Devils,ASU,Pac-10,NaN,Pac-12,823.0,429.0,370.0,Arizona State,Sun Devils,ASU,Pac-10,NaN,11,8,3,0


In [11]:
# subset the necessary columns and clean up data
playCallSuccess = merge_2[["season", "id", "team_x", "totalPlays", "passingPlays", "rushingPlays", "games", "wins", "losses"]]

playCallSuccess = playCallSuccess.rename(columns = {"team_x": "team"})

,season,id,team,totalPlays,passingPlays,rushingPlays,games,wins,losses
0,2004,2005,Air Force,147.0,33.0,110.0,11,5,6
1,2004,2006,Akron,131.0,70.0,55.0,11,6,5
2,2004,333,Alabama,677.0,224.0,434.0,11,6,5
3,2004,12,Arizona,681.0,314.0,354.0,11,3,8
4,2004,9,Arizona State,823.0,429.0,370.0,11,8,3
